In [11]:
import pandas as pd
import requests, json
import Utils as utils
from IPy import IP

servers = ['1.1.1.1', '8.8.8.8', '9.9.9.9']
ping_type = ['icmp', 'tcp']

In [12]:
df = utils.getAvailablePCProbesCC()

df = df.loc[df['CountryCode'].isin(utils.ccs)]
    
available_ccs = (df.groupby(['CountryCode']).size().to_frame(name = 'count').reset_index()).CountryCode.values
available_ccs

array(['AO', 'BW', 'CI', 'DZ', 'EG', 'GH', 'GQ', 'KE', 'MA', 'MG', 'MU',
       'NG', 'SD', 'SL', 'SN', 'TD', 'TN', 'UG', 'ZA', 'ZM', 'ZW'],
      dtype=object)

### First run some pings from all countries in Africa using ICMP to the list of public DNS resolvers
- 1.1.1.1
- 8.8.8.8
- 9.9.9.9

In [ ]:
df = pd.DataFrame(columns=['testid', 'cc', 'server', 'ping_type'])

for cc in utils.ccs:
    
    for s in servers:
        
        testID = utils.startPingTest(cc, s, 'icmp', 0, 0, 0)
        
        print(cc + ',' + s + ',' + testID)
        
        df = df.append({
            'testid': testID,
            'cc': cc,
            'server': s,
            'ping_type': 'icmp'
        }, ignore_index=True)

df.to_csv('data/pingTestID-28-06-2019.csv', encoding='utf8', index=False)

#### retrieve ping results

In [24]:
df = pd.read_csv('data/pingTestID-28-06-2019.csv')
df_pingres = pd.DataFrame(columns=['ASN',
                                   'ConnectionType',
                                   'CountryCode',
                                   'DNSResolver',
                                   'IPAddress',
                                   'Network',
                                   'ProbeID',
                                   'TestDateTime',
                                   'IPTarget',
                                   'rtt1',
                                   'rtt2',
                                   'rtt3',
                                   'avg_rtt'
                                  ])
for index, test in df.iterrows():
    res = utils.retrievePingTestResult(test.testid)
    for r in res:
        
        rtt1 = None
        rtt2 = None
        rtt3 = None
            
        try:
            rtt1 = r['PingArray'][0]
            rtt2 = r['PingArray'][1]
            rtt3 = r['PingArray'][2]
        except IndexError as e:
            continue
        except TypeError as e1:
            continue
        
        d = {'ASN': r['ProbeInfo']['ASN'],
             'ConnectionType': r['ProbeInfo']['ConnectionType'], 
             'CountryCode': r['ProbeInfo']['CountryCode'], 
             'DNSResolver': r['ProbeInfo']['DNSResolver'], 
             'IPAddress': r['ProbeInfo']['IPAddress'], 
             'Network': r['ProbeInfo']['Network'], 
             'ProbeID': r['ProbeInfo']['ProbeID'], 
             'TestDateTime': r['TestDateTime'], 
             'IPTarget': r['IP'], 
             'rtt1': r['PingArray'][0], 
             'rtt2': r['PingArray'][1], 
             'rtt3': r['PingArray'][2], 
             'avg_rtt': r['PingTime']}
        
        df_pingres = df_pingres.append(d, ignore_index=True)
df_pingres.to_csv('data/pingtests-28-06-2019.csv', sep=',', encoding='utf-8', index=False)   

### Run DIG tests locally and on the 3 different remote servers

In [25]:
df = pd.DataFrame(columns=['testid', 'cc', 'server', 'dig_type'])

for cc in available_ccs:
    
    testID = utils.startDigTest(cc, 'i.root-servers.net.', 0, 'IN', 'A', 0, 0, '', 0)
        
        #print(cc + ',' + s + ',' + testID)
        
    df = df.append({
        'testid': testID,
        'cc': cc,
        'server': '',
        'dig_type': 'local'
    }, ignore_index=True)
    
    for s in servers:
        
        testID = utils.startDigTest(cc, 'i.root-servers.net.', 0, 'IN', 'A', 0, 0, s, 0)
        
        #print(cc + ',' + s + ',' + testID)
        
        df = df.append({
            'testid': testID,
            'cc': cc,
            'server': s,
            'dig_type': 'remote'
        }, ignore_index=True)

df.to_csv('data/digTestID-28-06-2019.csv', encoding='utf8', index=False)

#### Retrieve DIG results

In [26]:
df = pd.read_csv('data/digTestID-28-06-2019.csv')
df_digres = pd.DataFrame(columns=['ASN',
                                   'ConnectionType',
                                   'CountryCode',
                                   'DNSResolver',
                                   'IPAddress',
                                   'Network',
                                   'ProbeID',
                                   'TestDateTime',
                                   'Server',
                                   'QueryTime',
                                   'Status',
                                   'dig_type'
                                  ])

for index, test in df.iterrows():
    res = utils.retrieveDigTestResult(test.testid)
    for r in res:
        
        d = {'ASN': r['ProbeInfo']['ASN'],
             'ConnectionType': r['ProbeInfo']['ConnectionType'], 
             'CountryCode': r['ProbeInfo']['CountryCode'], 
             'DNSResolver': r['ProbeInfo']['DNSResolver'], 
             'IPAddress': r['ProbeInfo']['IPAddress'], 
             'Network': r['ProbeInfo']['Network'],
             'ProbeID': r['ProbeInfo']['ProbeID'],
             'TestDateTime': r['TestDateTime'], 
             'Server': test.server,
             'QueryTime': r['QueryTime'],
             'Status' : r['Status'],
             'dig_type': test.dig_type
             }
        
        df_digres = df_digres.append(d, ignore_index=True)
df_digres.to_csv('data/digtests-28-06-2019.csv', sep=',', encoding='utf-8', index=False)   

### Run traceroute to the 3 remote servers

In [27]:
df = pd.DataFrame(columns=['testid', 'cc', 'server'])

for cc in utils.ccs:
    
    for s in servers:
        
        testID = utils.startTracertTest(cc, s)
                
        df = df.append({
            'testid': testID,
            'cc': cc,
            'server': s
        }, ignore_index=True)

df.to_csv('data/tracertTestID-28-06-2019.csv', encoding='utf8', index=False)

In [28]:
df = pd.read_csv('data/tracertTestID-28-06-2019.csv')
df_tracertres = pd.DataFrame(columns=[#'trace_id',
                                  'ASN',
                                  'ConnectionType',
                                   'CountryCode',
                                   'DNSResolver',
                                   'srcIPAddress',
                                   'Network',
                                   'Platform',
                                   'ProbeID',
                                   'TestDateTime',
                                   'dstIPAddress',
                                   'HostName',
                                   'hop_id',
                                   'hop_name',
                                   'hop_ip',
                                   'rtt1',
                                   'rtt2',
                                   'rtt3'
                                  ])

#trace_id=0
for index, test in df.iterrows():
    #trace_id +=1
    res = utils.retrieveTracertTestResult(test.testid)
        
    if (res is None):
            continue
    
    for r in res:
        
        hop_id=0
        
        for t in r['Tracert']:
            hop_id += 1
            
            try:
                rtt1 = t['PingTimeArray'][0]
                rtt2 = t['PingTimeArray'][1]
                rtt3 = t['PingTimeArray'][2]
            except IndexError as e:
                continue
            except TypeError as e1:
                continue
            
            d = {#'trace_id': trace_id,
                 'ASN': r['ProbeInfo']['ASN'],
                 'ConnectionType': r['ProbeInfo']['ConnectionType'], 
                 'CountryCode': r['ProbeInfo']['CountryCode'], 
                 'DNSResolver': r['ProbeInfo']['DNSResolver'], 
                 'srcIPAddress': r['ProbeInfo']['IPAddress'], 
                 'Network': r['ProbeInfo']['Network'],
                 'Platform': r['ProbeInfo']['Platform'],
                 'ProbeID': r['ProbeInfo']['ProbeID'],
                 'TestDateTime': r['TestDateTime'], 
                 'dstIPAddress': r['Destination'],
                 'HostName': r['HostName'],
                 'hop_id': hop_id,
                 'hop_name': t['HostName'],
                 'hop_ip': t['IP'],
                 'rtt1': rtt1,
                 'rtt2': rtt2,
                 'rtt3': rtt3
                 }
        
            df_tracertres = df_tracertres.append(d, ignore_index=True)
df_tracertres.to_csv('data/tracertTests-28-06-2019.csv', sep=',', encoding='utf-8', index=False)   

#### Run traceroute from probe to DNSResolvers

In [41]:
df_tracertres = pd.read_csv('data/tracertTests-26-06-2019.csv')
df_tracertres = df_tracertres[df_tracertres.DNSResolver.notnull()]

df1 = pd.DataFrame(columns=['testid', 'cc', 'server'])

for index, probe in df_tracertres.iterrows():
    probeid = probe.ProbeID
    dnsresolver = probe.DNSResolver
    
    if (IP(dnsresolver).iptype() == 'PRIVATE'):
        continue
    
    try:
        testID = utils.startTracertTestByProbe(probeid, dnsresolver)
    except JSONDecodeError as e:
        continue
    
    df1 = df1.append({
            'testid': testID,
            'probeid': probeid,
            'server': dnsresolver
        }, ignore_index=True)

df1.to_csv('data/tracert-resolver-TestID-26-06-2019.csv', encoding='utf8', index=False)

In [42]:
df = pd.read_csv('data/tracert-resolver-TestID-26-06-2019.csv')

df_tracertres = pd.DataFrame(columns=[#'trace_id',
                                  'ASN',
                                  'ConnectionType',
                                   'CountryCode',
                                   'DNSResolver',
                                   'srcIPAddress',
                                   'Network',
                                   'Platform',
                                   'ProbeID',
                                   'TestDateTime',
                                   'dstIPAddress',
                                   'HostName',
                                   'hop_id',
                                   'hop_name',
                                   'hop_ip',
                                   'rtt1',
                                   'rtt2',
                                   'rtt3'
                                  ])

#trace_id=0
for index, test in df.iterrows():
    #trace_id +=1
    if (test.testid == 'FAILED'):
        continue
    
    res = utils.retrieveTracertTestResult(test.testid)
        
    if (res is None):
            continue
    
    for r in res:
        
        if (r['Tracert'] is None):
            continue
        
        hop_id=0
        
        for t in r['Tracert']:
                
            hop_id += 1
            
            try:
                rtt1 = t['PingTimeArray'][0]
                rtt2 = t['PingTimeArray'][1]
                rtt3 = t['PingTimeArray'][2]
            except IndexError as e:
                continue
            except TypeError as e1:
                continue
            
            d = {#'trace_id': trace_id,
                 'ASN': r['ProbeInfo']['ASN'],
                 'ConnectionType': r['ProbeInfo']['ConnectionType'], 
                 'CountryCode': r['ProbeInfo']['CountryCode'], 
                 'DNSResolver': r['ProbeInfo']['DNSResolver'], 
                 'srcIPAddress': r['ProbeInfo']['IPAddress'], 
                 'Network': r['ProbeInfo']['Network'],
                 'Platform': r['ProbeInfo']['Platform'],
                 'ProbeID': r['ProbeInfo']['ProbeID'],
                 'TestDateTime': r['TestDateTime'], 
                 'dstIPAddress': r['Destination'],
                 'HostName': r['HostName'],
                 'hop_id': hop_id,
                 'hop_name': t['HostName'],
                 'hop_ip': t['IP'],
                 'rtt1': rtt1,
                 'rtt2': rtt2,
                 'rtt3': rtt3
                 }
        
            df_tracertres = df_tracertres.append(d, ignore_index=True)
df_tracertres.to_csv('data/tracertTests-resolver-26-06-2019.csv', sep=',', encoding='utf-8', index=False)   

In [ ]:
utils.retrieveTracertTestResult('611ec2d9-24b6-4cb9-9ca3-7f433f18e5fe')